In [1]:
import uproot

from analysis_simulation import Analysis_simulation, magnitude

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
import seaborn as sn
from tqdm import tqdm

import numpy as np
from numpy import array, where, shape, reshape, pi, cos, sin, sqrt, linspace
np.set_printoptions(threshold=1000)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# 1) Mapping all PMT

In [2]:
file = uproot.open("/sno/py_out1.root")

### Simulation data

In [3]:
data1 = file['T;7']
data1.keys()

['evtid',
 'mc_position',
 'mc_momentum',
 'position',
 'momentum',
 'hit_pmtid',
 'hit_pmttime',
 'hit_residual',
 'hit_type']

In [4]:
evtid = np.array(data1['evtid'])
mc_position = np.array(data1['mc_position'])
position = np.array(data1['position'])
hit_pmtid = np.array(data1['hit_pmtid'])
hit_pmttime = np.array(data1['hit_pmttime'])
hit_residual = np.array(data1['hit_residual'])
hit_type = np.array(data1['hit_type'])

### PMT info.

In [5]:
pmt_info = file['pmt;1']
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

In [6]:
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])

### Separate PMT type coordinates

In [7]:
#Lets separate their coordinates
#xyz:

prefix_xyz = 'type_'
suffix_xyz = '_xyz'

type_var = np.unique(pmt_type)

for i in type_var:
    condition = (pmt_type == i)
    locals()[prefix_xyz + str(i) + suffix_xyz] = []
    
    for j in (where(condition)[0]):
        locals()[prefix_xyz + str(i) + suffix_xyz].append(pmt_pos_xyz[j])
    locals()[prefix_xyz + str(i) + suffix_xyz] = np.array(locals()[prefix_xyz + str(i) + suffix_xyz])
    
#spherical:

prefix_sph = 'type_'
suffix_sph = '_sph'

type_var = np.unique(pmt_type)

for i in type_var:
    condition = (pmt_type == i)
    locals()[prefix_sph + str(i) + suffix_sph] = []
    
    for j in (where(condition)[0]):
        locals()[prefix_sph + str(i) + suffix_sph].append(pmt_pos_sph[j])
    locals()[prefix_sph + str(i) + suffix_sph] = np.array(locals()[prefix_sph + str(i) + suffix_sph])

## 1.1) Create PMT distribution in $\theta\phi$ plane

In [8]:
zenit = type_1_sph[:,0]
azimut = type_1_sph[:,1]

In [15]:
%matplotlib

plt.figure(figsize = (10,7))
plt.hist2d(x = np.cos(zenit), y = (azimut), bins = [100,100])
plt.ylabel('azimuth')
plt.xlabel('cos(zenith)')
plt.title('All PMT type = 1 Map')
#plt.colorbar()
plt.show()
plt.savefig('All PMT type1 Map.jpg')

Using matplotlib backend: Qt5Agg


In [11]:
#Transformtion of zenith

zenit_T = zenit + np.pi/2

In [23]:
plt.figure(figsize = (10,7))
plt.hist2d(x = np.cos(zenit+pi/2), y = (azimut), bins = [50,100])
plt.ylabel('azimuth')
plt.xlabel('cos(zenith)')
plt.title('All PMT type = 1 Map Transformed')
#plt.colorbar()
plt.show()

# 2) Tratement of and event

In [16]:
df = Analysis_simulation(file, [11,20,50])

In [24]:
df

,eventID,hitpmt ID,hit type,mc coordinates,mc radius,reconst radius,reconst error
0,11,"[2579, 8519, 8484, 8966, 8511, 8507, 8486, 897...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[3557.810791015625, -2256.099365234375, 3368.9...",5394.239748,5421.818817,48.522873
1,20,"[8360, 8243, 8780, 8344, 8352, 8266, 8849, 835...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1800.2008056640625, -1955.141845703125, -4592...",5305.681847,5356.561970,112.391825
2,50,"[3101, 5526, 4806, 5302, 5200, 6218, 6237, 517...","[2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, ...","[-1761.8115234375, -815.3781127929688, -1181.5...",2272.642333,2406.404738,229.281677
